# Multitaper with gaps 

It is possible to compute the multitaper spectrum on data that contain large gaps. In this notebook we demo the capabilities of this functionality.

In [ ]:
# Bring in the testfile
using DelimitedFiles, Multitaper, Plots
pyplot()

dat = readdlm("./data/temp.txt", ',')

tt = dat[:,1]
xx = dat[:,2]

plot(tt,xx)

We're going to make a set of slepian tapers having a gap. To see these, use


In [ ]:
NW = 4.0
W = NW/length(tt)
egvals,sleps = MDslepian(W, 6, tt)
plot(tt,sleps)

There is one gap in the data between t = 101 and t = 256. To do the spectral analysis, use

In [ ]:
out = MDmwps(tt,xx,alpha=0.99)

The first output will simply be the estimated spectrum. By default however, this code computes the F-test statistic as well.

In [ ]:
plot(out,c=1)

These are the p-values for the F-test statistic. I've scattered it where there are significant line components as indicated by the F-test result. The alpha value for the test here is small, set to the reciprocal of the length of the series.

In [ ]:
plot(out.f, out.Fpval, label = "F-test p-value")
ind = findall(out.Fpval .< 1/length(tt))
println("Significance level set to $(1.0-1/length(tt)).")
scatter!(out.f[ind], out.Fpval[ind], label="Significant peaks")

There is nothing deep about this example, in that the data were simply white noise with added gaps, so there is not much to be said about this result. 

# References

Chave, Alan D. "A multitaper spectral estimator for time-series with missing data." Geophysical Journal International 218.3 (2019): 2165-2178.
